<a href="https://colab.research.google.com/github/hdakhli/mlops-2024/blob/main/20_neural_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classifier des images

Suivre les étapes indiquées dans le notebook pour classifier des images de chiffres manuscrits en utilisant un réseau de neurones.

<br>

<img src="datasets/MNIST-dataset.png">

In [16]:
!pip install numpy tensorflow matplotlib opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 2.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 2.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.2
    Uninstalling numpy-1.26.2:
      Successfully uninstalled numpy-1.26.2


### Importer les librairies

- Tensorflow is an end-to-end open source platform for machine learning. 
- Keras is an open-source library that provides a Python interface for artificial neural networks.
- numpy is the main package for scientific computing with Python.
- matplotlib is a library to plot graphs in Python.

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation
np.random.seed(42)
from keras.optimizers import Adam, SGD      # help us to transform our data later
from tensorflow.python.keras.utils import np_utils

2024-05-27 10:55:44.649738: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load the MNIST dataset

In [4]:
#(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
import ssl
import urllib.request

ssl._create_default_https_context = ssl._create_unverified_context
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 6s 0us/step


### Checkout input dataset

In [5]:
print('Total no of Images: ',X_train.shape[0])
print('Size of Image:', X_train.shape[1:])
print('Total no of labels:', y_train.shape)

Total no of Images:  60000
Size of Image: (28, 28)
Total no of labels: (60000,)


### Prepare input data
For a multi-layer perceptron model we must reduce the images down into a vector of pixels. In this case the 28×28 sized images will be 784 pixel input values.

We can do this transform easily using the reshape() function on the NumPy array. We can also reduce our memory requirements by forcing the precision of the pixel values to be 32 bit.

In [6]:
X_train = X_train.reshape((X_train.shape[0],-1))
X_test = X_test.reshape((X_test.shape[0], -1))

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print(X_train.shape, X_test.shape)

(60000, 784) (10000, 784)


### Normalize Input data
The pixel values are gray scale between 0 and 255. It is almost always a good idea to perform some scaling of input values when using neural network models. Because the scale is well known and well behaved, we can very quickly normalize the pixel values to the range 0 and 1 by dividing each value by the maximum of 255.
<br>
<img src="datasets/three_pixels.png" width="800">

In [7]:
X_train = X_train/255
X_test = X_test/255

# print(X_train[0])
X_train.shape

(60000, 784)

## One-hot Encoding

Finally, the output variable is an integer from 0 to 9. This is a multi-class classification problem. As such, it is good practice to use a one hot encoding of the class values, transforming the vector of class integers into a binary matrix.

### What is Categorial Data?

Categorical data are variables that contain label values rather than numeric values. The number of possible values is often limited to a fixed set. 

Some examples include:

- A “pet” variable with the values: “dog” and “cat“.
- A “color” variable with the values: “red“, “green” and “blue“.
- A “place” variable with the values: “first”, “second” and “third“.
- A "Number" variable with values: "0", "1", "2", ... , "9".

### Convert Categorical Data to Numerical Data.

Here, the integer encoded variable is removed and a new binary variable is added for each unique integer value.


In the “color” variable example, there are 3 categories and therefore 3 binary variables are needed. A “1” value is placed in the binary variable for the color and “0” values for the other colors.

    - red,		green,		blue
    - 1,		0,		0
    - 0,		1,		0
    - 0,		0,		1


We can easily do this using the built-in **np_utils.to_categorical()** helper function in Keras.

In [8]:
# One-hot encoding

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

print(y_train.shape)

(60000, 10)


In [9]:
num_classes = y_test.shape[1]
num_pixels = 784

### Define Model

Input size of our model - 784 (28 x 28). There are 2 hidden layers with 256 and 64 neurons in each.
In hidden layers, we will use relu activation function. And in final output layer, we'll use softmax activation function.

#### Relu Function:
The rectified linear activation function or ReLU for short is a piecewise linear function that will output the input directly if it is positive, otherwise, it will output zero.

<br>

#### Softmax Function:
The Softmax function that turns numbers aka logits into probabilities that sum to one. Softmax function outputs a vector that represents the probability distributions of a list of potential outcomes.

<br>

<img src="datasets/activation.png" width="1000">

<br>

So our final model look like this,

<br>

<img src="datasets/nn_archi.png" width="1000">



In [10]:
# define baseline model
model = Sequential()
model.add(Dense(256, input_dim=num_pixels, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(num_classes, activation='softmax'))

In [11]:
# build the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               200960    
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 218,058
Trainable params: 218,058
Non-trainable params: 0
_________________________________________________________________


### Compile Model

#### Loss Function: Categorical Crossentropy
The categorical crossentropy is well suited to classification tasks, since one example can be considered to belong to a specific category with probability 1, and to other categories with probability 0.

<br/>

#### Optimizer: SGD (Stochastic Gradient Descent)
Stochastic gradient descent is an iterative method for optimizing an objective function with suitable smoothness properties

In [12]:
opt = SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

### Train Model
To fit the model, all we have to do is declare the batch size and number of epochs to train for, then pass in our training data.

In [13]:
model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1)

Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 1.7253 - accuracy: 0.5628
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.8591 - accuracy: 0.8170
Epoch 3/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.5806 - accuracy: 0.8574
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.4764 - accuracy: 0.8759
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.4218 - accuracy: 0.8859


In [14]:
scores = model.evaluate(X_test, y_test, verbose=1)

313/313 [==============================] - 2s 6ms/step - loss: 0.3849 - accuracy: 0.8938


In [17]:
# Faire une prédiction d'une image de test
P = model.predict(X_test)
for k in range(len(P[1])):
    if P[1][k]==P[1].max():
        print(k)
img = Image.fromarray(X_test)
img.show()
# Tester avec les images présentes dans le dossier "datasets"

313/313 [==============================] - 2s 6ms/step
2


NameError: name 'Image' is not defined